In [1]:
ls = sc.textFile('../../Data/spark/coupon150720.csv')

In [2]:
ls.take(3)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0']

In [6]:
ls.getNumPartitions()

3

### Ingreso total por aerolinea

In [3]:
def getAlIng(l):
    elems = l.split(",")
    al = elems[4]
    ingreso = float(elems[6])
    return (al, ingreso)

In [13]:
ls.map(getAlIng).groupByKey().mapValues(lambda x: sum(x)).sortBy(lambda x: x[1], False).take(10)

[(u'GA', 20712226.56999984),
 (u'BA', 12443986.980000038),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.900000253),
 (u'LH', 7715608.339999713),
 (u'QR', 6935584.369999946),
 (u'SV', 6286299.340001837),
 (u'UA', 5151815.289999981),
 (u'JJ', 5066300.509999913),
 (u'AA', 4151996.4599999674)]

In [16]:
ls.map(getAlIng).reduceByKey(lambda e,acum: e+acum).sortBy(lambda x: x[1],False).take(10)

[(u'GA', 20712226.569999855),
 (u'BA', 12443986.980000002),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.9000001205),
 (u'LH', 7715608.339999718),
 (u'QR', 6935584.369999997),
 (u'SV', 6286299.340001843),
 (u'UA', 5151815.29),
 (u'JJ', 5066300.509999905),
 (u'AA', 4151996.4599999613)]

In [7]:
def getALIngreso(l):
    elems = l.split(",")
    al = elems[4]
    ingreso = float(elems[6])
    return (al, ingreso)

In [8]:
getALIngreso(ls.first())

(u'9W', 56.79)

In [9]:
als = ls.map(getALIngreso)

In [10]:
als.take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [11]:
als.groupByKey().take(2)

[(u'', <pyspark.resultiterable.ResultIterable at 0x7f00b98d22d0>),
 (u'BE', <pyspark.resultiterable.ResultIterable at 0x7f00b98d2490>)]

In [18]:
totales = als.groupByKey().mapValues(lambda vs: sum(vs))

In [20]:
totales.sortBy(lambda x: x[1], False).take(10)

[(u'GA', 20712226.56999984),
 (u'BA', 12443986.980000038),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.900000253),
 (u'LH', 7715608.339999713),
 (u'QR', 6935584.369999946),
 (u'SV', 6286299.340001837),
 (u'UA', 5151815.289999981),
 (u'JJ', 5066300.509999913),
 (u'AA', 4151996.4599999674)]

#### Lo mismo con reduceByKey

In [23]:
als.reduceByKey(lambda e,acum: e+acum).sortBy(lambda x: x[1], False).take(10)


[(u'GA', 20712226.569999855),
 (u'BA', 12443986.980000002),
 (u'AF', 10221507.909999428),
 (u'QF', 8358587.9000001205),
 (u'LH', 7715608.339999718),
 (u'QR', 6935584.369999997),
 (u'SV', 6286299.340001843),
 (u'UA', 5151815.29),
 (u'JJ', 5066300.509999905),
 (u'AA', 4151996.4599999613)]

### Top 5 de ingreso medio por pasajero con reduceByKey

In [34]:
grup = ls.map(getAlIng).map(lambda x: (x[0], (x[1],1))).reduceByKey(lambda x,acum: (x[0]+acum[0], x[1]+acum[1]))

In [37]:
grup.mapValues(lambda x: x[0]/x[1]).sortBy(lambda x: x[1],False).take(5)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.6739864864867),
 (u'GA', 991.2527671691723),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035)]

In [45]:
als.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)).sortBy(lambda x: x[1], False).take(5)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.673986486487),
 (u'GA', 991.2527671691715),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035)]

In [58]:
def calculaMediaReduce(e, acum):
    sumaTotal = acum[0]
    numElems = acum[1]
    return (e[0] + sumaTotal, e[1] + numElems)

In [64]:
tr = als.map(lambda x: (x[0], (x[1], 1))).reduceByKey(calculaMediaReduce)

In [65]:
tr.take(2)

[(u'', (478.31000000000006, 18811)), (u'BE', (64073.32999999998, 788))]

In [66]:
tr.mapValues(lambda x: x[0]/x[1]).sortBy(lambda x: x[1], False).take(5)

[(u'S3', 5225.068852459017),
 (u'9V', 1488.6739864864867),
 (u'GA', 991.2527671691723),
 (u'TN', 964.6472580645161),
 (u'7F', 668.035)]